In [2]:
import cv2
import numpy as np
import os
from tensorflow import keras 
from keras.preprocessing import image

In [3]:
categories=['without_mask','with_mask']

In [4]:
os.getcwd()
os.chdir(r"C:\Users\yedla\Dropbox\PC\Documents\DATA_SCI.ML\Project\data")
curr_path= os.getcwd()
curr_path

'C:\\Users\\yedla\\Dropbox\\PC\\Documents\\DATA_SCI.ML\\Project\\data'

In [5]:
data=[]
for category in categories:
  path=os.path.join(curr_path,category)
  label= categories.index(category)

  for file in os.listdir(path):
    img_path=os.path.join(path,file)
# lets read the image 
    img=cv2.imread(img_path)
#since VGG16 takes 224,224 shape image we need to resize the image
    img=cv2.resize(img,(224,224))
    data.append([img,label])

In [6]:
len(data)


1376

In [10]:
#shuffle the data
import random
random.shuffle(data)

In [11]:
x=[]
y=[] 
for images, labels in data:
  x.append(images) #images are stored in x variable
  y.append(labels) # labels are stored in y

In [12]:
print(len(x))
print(len(y))

1376
1376


In [13]:
# x and y are in list formats. Lets convert into numpy array
x=np.array(x)
y=np.array(y)

In [14]:
print(x.shape,y.shape) 

(1376, 224, 224, 3) (1376,)


In [15]:
y[:-4]

array([1, 1, 1, ..., 1, 0, 0])

In [16]:
#lets preprocess input image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

In [17]:
x=x/255
x[0]

array([[[0.52941176, 0.69411765, 0.85882353],
        [0.52941176, 0.69411765, 0.85882353],
        [0.5254902 , 0.69019608, 0.85882353],
        ...,
        [0.7254902 , 0.76862745, 0.8745098 ],
        [0.7254902 , 0.76862745, 0.8745098 ],
        [0.7254902 , 0.76862745, 0.8745098 ]],

       [[0.51764706, 0.68235294, 0.85098039],
        [0.51764706, 0.68235294, 0.85098039],
        [0.52156863, 0.68235294, 0.85490196],
        ...,
        [0.7254902 , 0.76862745, 0.8745098 ],
        [0.7254902 , 0.76862745, 0.8745098 ],
        [0.7254902 , 0.76862745, 0.8745098 ]],

       [[0.50980392, 0.6745098 , 0.84705882],
        [0.50980392, 0.6745098 , 0.84705882],
        [0.51764706, 0.6745098 , 0.85098039],
        ...,
        [0.7254902 , 0.76862745, 0.8745098 ],
        [0.7254902 , 0.76862745, 0.8745098 ],
        [0.7254902 , 0.76862745, 0.8745098 ]],

       ...,

       [[0.23529412, 0.27058824, 0.62352941],
        [0.23137255, 0.26666667, 0.61960784],
        [0.22745098, 0

In [18]:
vgg=VGG16()
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [19]:
model=keras.Sequential()

In [20]:
for layer in vgg.layers[:-1]:
  model.add(layer) #last layer is removed

In [21]:
# making parameters as non-trainable
for layer in model.layers:
  layer.trainable=False

In [22]:
#adding a output layer
model.add(keras.layers.Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [23]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics='accuracy')

In [24]:
#now that model architecture is ready
#lets split the data
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.2,random_state=42)

In [25]:
print(x_train.shape, x_test.shape)

(1100, 224, 224, 3) (276, 224, 224, 3)


In [26]:
#lets train the model
model.fit(x_train,y_train, epochs=5)

Epoch 1/5
35/35 [==============================] - 190s 5s/step - loss: 0.6647 - accuracy: 0.5927
Epoch 2/5
35/35 [==============================] - 188s 5s/step - loss: 0.3770 - accuracy: 0.9036
Epoch 3/5
35/35 [==============================] - 186s 5s/step - loss: 0.2877 - accuracy: 0.9273
Epoch 4/5
35/35 [==============================] - 201s 6s/step - loss: 0.2458 - accuracy: 0.9309
Epoch 5/5
35/35 [==============================] - 202s 6s/step - loss: 0.1973 - accuracy: 0.9536


In [27]:
def face_mask(img):
    img= cv2.resize(img,(224,224))
    img=img.reshape(1,224,224,3)
    y_pred= model.predict(img)
    return y_pred[0][0]

In [28]:
sampleimg1= cv2.imread(r"C:\Users\yedla\Dropbox\PC\Documents\DATA_SCI.ML\Project\data\download1.jpeg")#with_mask
sampleimg2=cv2.imread(r"C:\Users\yedla\Dropbox\PC\Documents\DATA_SCI.ML\Project\data\image3.jpeg")# without_msk

In [30]:
face_mask(sampleimg2)

1/1 [==============================] - 1s 964ms/step


0.97194445

In [31]:
cap=cv2.VideoCapture(0)

In [53]:
haar_cascade= cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [54]:
def detect_face(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    coords=haar_cascade.detectMultiScale(gray_img,1.1,9)
    return coords

In [55]:
def draw_label(img,text,pos,bg_color):
    text_size= cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,cv2.FILLED)
    
    end_x= pos[0]+text_size[0][0]+2
    end_y= pos[1] +text_size[0][1]-2
    
    cv2.rectangle(img,pos,(end_x,end_y),bg_color,cv2.FILLED)
    cv2.putText(img,text,pos,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),1,cv2.LINE_AA)

In [56]:
while True:
    ret,frame= cap.read()
    img= cv2.resize(frame,(224,224))
    #detect_face_mask
    
    y_pred=face_mask(img)
    # draw a rectangle around the coords
    coords=detect_face(img)
    for x,y,w,h in coords:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),3)
    
    if y_pred <0.8:
        draw_label(frame,"No Mask",(30,30),(0,0,255))
    else:
        draw_label(frame,"Mask",(30,30),(0,0,255))
    
    cv2.imshow("Window",frame)
    
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
cv2.destroyAllwindows()
    

1/1 [==============================] - 0s 179ms/step


1/1 [==============================] - 0s 173ms/step


AttributeError: module 'cv2' has no attribute 'destroyAllwindows'